## 1.List all files in the folder

In [22]:
import os

folder_path = './benchmark/contracts/22'

sol_files_info = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.sol'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                file_content = f.read()
            sol_files_info.append({
                'name': file,
                'path': file_path
            })
print(f"Number of files in the folder: : {len(sol_files_info)}")
print(f"List all files in the folder {sol_files_info}")
  

Number of files in the folder: : 43
List all files in the folder [{'name': 'Staker.sol', 'path': './benchmark/contracts/22/contracts/contracts/Staker.sol'}, {'name': 'Migrations.sol', 'path': './benchmark/contracts/22/contracts/contracts/Migrations.sol'}, {'name': 'LongShort.sol', 'path': './benchmark/contracts/22/contracts/contracts/LongShort.sol'}, {'name': 'FloatCapital_v0.sol', 'path': './benchmark/contracts/22/contracts/contracts/FloatCapital_v0.sol'}, {'name': 'YieldManagerAave.sol', 'path': './benchmark/contracts/22/contracts/contracts/YieldManagerAave.sol'}, {'name': 'SyntheticToken.sol', 'path': './benchmark/contracts/22/contracts/contracts/SyntheticToken.sol'}, {'name': 'FloatToken.sol', 'path': './benchmark/contracts/22/contracts/contracts/FloatToken.sol'}, {'name': 'TokenFactory.sol', 'path': './benchmark/contracts/22/contracts/contracts/TokenFactory.sol'}, {'name': 'Treasury_v0.sol', 'path': './benchmark/contracts/22/contracts/contracts/Treasury_v0.sol'}, {'name': 'Dai.sol

## 2. Run BA mode of LLM-SmartAudit on each project

In [ ]:
import os
import subprocess

os.environ['OPENAI_API_KEY'] = ''

# Specify project you want to detect
project_file = 18

# Specify the folder path
folder_path = f'./benchmark/contracts/{project_file}'

files = os.listdir(folder_path)

# Function to run the command for a given file
def run_command(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        file_content = f.read()
    
    command = [
        "python", "run.py",
        "--org", "RealWord",
        "--config", "SmartContractBA",
        "--task", file_content,
        "--name", f"RealWorld_BA_{project_file}_{os.path.splitext(os.path.basename(file_path))[0]}"
    ]
    
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print(f"Successfully ran {file_path}")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error running {file_path}")
        print(e.stderr)

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.sol'):
            file_path = os.path.join(root, file)
            print(f"Processing {file_path}...")
            run_command(file_path)
        else:
            print(f"Skipping {file} (not a .sol file)")

## 3. Run TA mode of LLM-SmartAudit on each project

In [ ]:
import os
import subprocess

os.environ['OPENAI_API_KEY'] = ''

# Specify project you want to detect
project_file = 25

# Specify the folder path
folder_path = f'./benchmark/contracts/{project_file}'

# List all files in the folder
files = os.listdir(folder_path)

# Function to run the command for a given file
def run_command_TA(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        file_content = f.read()
    
    command = [
        "python", "run.py",
        "--org", "RealWorld",
        "--config", "SmartContractTA",
        "--task", file_content,
        "--name", f"RealWorld_TA_{project_file}_{os.path.splitext(file)[0]}"
    ]
    
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print(f"Successfully ran {file}")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error running {file}")
        print(e.stderr)

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.sol'):
            file_path = os.path.join(root, file)
            print(f"Processing {file_path}...")
            run_command_TA(file_path)
        else:
            print(f"Skipping {file} (not a .sol file)")

## 4. Zero-shot Prompting

In [ ]:
import os
import csv
from openai import OpenAI
import pandas as pd

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

project_file = 'proc_arithmetic'

data = pd.read_csv('./datasets_curated.csv')

# Initialize OpenAI client
client = OpenAI()

def extract_content(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content


def run_command(content):
    try:
        # Adding the instruction before the content
        prompt = f"List all the vulnerabilities in the following smart contract:\n\n{content}"
        
        response = client.chat.completions.create(
            model="gpt-4",  # Use the appropriate model gpt-3.5-turbo, gpt-4o
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        
        output = response.choices[0].message.content
        
        return output
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Error processing the file"
    


def process_files_and_save_to_csv(folder_path, csv_output_path):
    with open(csv_output_path, 'w', newline='') as csvfile:
        fieldnames = ['file_name', 'content', 'output']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith('.sol'):
                    file_path = os.path.join(root, file)
                    print(f"Processing {file_path}...")
                    content = extract_content(file_path)
                    output = run_command(content)
                    # Write the file name, content, and output to the CSV
                    writer.writerow({'file_name': file, 'content': content, 'output': output})
                else:
                    print(f"Skipping {file} (not a .sol file)")

csv_output_path = './evaluation/LabeledResult/zero-shot-gpt35.csv' 

process_files_and_save_to_csv(folder_path, csv_output_path)